In [1]:
import pandas as pd

import re
import numpy as np
import pandas as pd
import sys 
from keras.models import Sequential
from keras.layers import LSTM, Activation,  Dropout, Dense, Embedding, Bidirectional

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import keras.utils as ku 

from keras.callbacks import ModelCheckpoint

import pickle

import random as rd

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/glep93/X-mas_song_generator/main/xmas-songs.csv')

In [3]:
def preprocessing(x):
    x = x.lower()
    x = re.sub('\[(.*?)\]', '', x)
    x = re.sub('[\n]{2,}', '\n',x)
    x = re.sub("[^a-z\'\s\n]",'',x)
    x = re.sub('[\n]', ' *newline* ',x)
    x = re.sub('^\s','',x)
    x = re.sub('[\s]{2,}', ' ',x)
    out =np.array( x.split(' ') )
    out = out[out!= '']
    return out 

In [4]:
lyrics =df['lyrics'].dropna()[ df.Artist=='Michael Bublé'][[13 ,170 ,289 ,387 ,592]]
lyrics = lyrics.apply(preprocessing )

In [5]:
aux = []
for i in lyrics:
    aux  = np.append(aux, i)

vocab = set( list(aux) )

index_to_word = { i+1: word for i, word in enumerate(vocab) }
word_to_index = {  word: i+1 for i, word in enumerate(vocab) }
index_to_word[0] = 'missing_word'
word_to_index['missing_word'] = 0

total_words = len( vocab)+1
print(total_words)

335


In [6]:
pickle.dump( index_to_word, open( "index_to_word.p", "wb" ) )
pickle.dump( word_to_index, open( "word_to_index.p", "wb" ) )

In [7]:
n = 30
X = []
y = []

for song in lyrics:
    N = len(song)
    for i in range( N -n- 1):
        for j in range(1,n):
            X.append(  [word_to_index[word]  for word in song[i:i+j]] )
            y.append( word_to_index[song[i+j]])


In [8]:
y = np.array(y)

In [9]:
X_pad = pad_sequences(X, maxlen = n, padding='pre')
X_pad.shape

(38889, 30)

In [10]:
#num_sample_training = 100000
#tot=len(X_pad)
#index_training=np.random.randint(0,tot,num_sample_training)

In [11]:
X_train = X_pad#[index_training]
y_train = y#[index_training]

In [12]:
model = Sequential()

# Add Input Embedding Layer
model.add(Embedding(total_words, 50, input_length=30))
    
 # Add Hidden Layer 1 - LSTM Layer
model.add(Bidirectional(LSTM(128, return_sequences=True)))
#model.add(LSTM(128))
model.add(Dropout(0.2))

model.add(LSTM(64))

model.add(Dense(total_words/2, activation='relu')) 

# Add Output Layer
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 30, 50)            16750     
_________________________________________________________________
bidirectional (Bidirectional (None, 30, 256)           183296    
_________________________________________________________________
dropout (Dropout)            (None, 30, 256)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                82176     
_________________________________________________________________
dense (Dense)                (None, 167)               10855     
_________________________________________________________________
dense_1 (Dense)              (None, 335)               56280     
Total params: 349,357
Trainable params: 349,357
Non-trainable params: 0
__________________________________________________

In [13]:
label = ku.to_categorical(y, num_classes=total_words)

In [16]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
history = model.fit(X_pad, label, epochs=5, batch_size=512, callbacks=callbacks_list)

Epoch 1/5
76/76 [==============================] - ETA: 0s - loss: 0.6407
Epoch 00001: loss improved from inf to 0.64074, saving model to weights-improvement-01-0.6407.hdf5
76/76 [==============================] - 23s 301ms/step - loss: 0.6407
Epoch 2/5
76/76 [==============================] - ETA: 0s - loss: 0.6112
Epoch 00002: loss improved from 0.64074 to 0.61117, saving model to weights-improvement-02-0.6112.hdf5
76/76 [==============================] - 26s 346ms/step - loss: 0.6112
Epoch 3/5
76/76 [==============================] - ETA: 0s - loss: 0.5426
Epoch 00003: loss improved from 0.61117 to 0.54259, saving model to weights-improvement-03-0.5426.hdf5
76/76 [==============================] - 26s 343ms/step - loss: 0.5426
Epoch 4/5
76/76 [==============================] - ETA: 0s - loss: 0.4990
Epoch 00004: loss improved from 0.54259 to 0.49902, saving model to weights-improvement-04-0.4990.hdf5
76/76 [==============================] - 26s 343ms/step - loss: 0.4990
Epoch 5/5
76

In [17]:
model.save('mAIchael_buble')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: mAIchael_buble/assets


In [18]:
from tensorflow import keras
model_load = keras.models.load_model('mAIchael_buble')

In [19]:
def make_lyrics( input, n_words):
    input_p = preprocessing(input)
    for i in range(n_words):
        input_pad = pad_sequences( [[word_to_index[word] if word in word_to_index.keys() else 0  for word in input_p ]] , maxlen = n, padding='pre')
        predict = model_load.predict_classes(input_pad, verbose=0)[0]
        if index_to_word[predict] == '*newline*':
            input = input + '\n'
        else:
            input = input +' '+ index_to_word[predict] 
        input_p = np.append(input_p, index_to_word[predict])
    return input

In [20]:
input  = '''all i want for christmas'''

print(make_lyrics( input, 200))

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
all i want for christmas
 is you
 and all the lights are shining
 so brightly everywhere
 and the sound of children's
 laughter fills the air
 and everyone is singing
 i can hear those sleigh bells ringing
 santa won't you bring me the one i really love
 i won't even wish for snow
 no i'm just gonna keep on waiting
 underneath the mistletoe
 there's no sense in hanging stockings
 there upon the fireplace
 'cause santa he won't make me happy
 with a toy on christmas day
 i just want you here
 let's give love and get some back
 the more you give the more you'll have
 the fire's bright now
 and snow comes down
 grab your family hold them near
 christmas time is

In [21]:
df['lyrics'].dropna()[ df.Artist=='Michael Bublé'][[13,144 ,170 ,289 ,387 ,592]]
print(df['lyrics'].dropna()[ df.Artist=='Michael Bublé'][144])

[Intro 1:  0ld 420]
:^|

[Intro 2: 
Yung McLovin
]
Shoutout to the real dope dealers forreal AHAAAAAA, shoutout to my mom and dad for the support, shoutout to Maboo for making this possible, shoutout to the real niqqas who smoke crack, shoutout to that neighbor that doesn’t try to talk to me when I’m walking out the building and he’s walking in, shoutout to bubble wrap, shoutout to the WNBA, shoutout to the teacher that doesn’t give homework on a friday, shoutout to the skippable YouTube ad, shoutout to ad blocker for making the previous shoutout irrelevant, shoutout to the kid that let me copy back in 8th grade when I forgot we had test, shoutout to piratebay, shoutout to kickass for being there for me after the piratebay ship sunk, shoutout to all the memes in chat, shoutout to the other bars below me, shoutout to the illuminati, shoutout to white devil sophistry; let’s fukken start this shit

[Bridge - Hoshi]
Let's get this going

[FIRST VERSE: God]
Flow so white you’d think I was a